In [ ]:
from IPython.display import clear_output
!pip install keras
!pip install tensorflow
clear_output()

In [ ]:
import os
import numpy as np
import pandas as pd
import cv2
import matplotlib.pyplot as plt
import seaborn as sns

os.listdir('../input/cassava-leaf-disease-classification')

In [ ]:
train_img_path = '../input/cassava-leaf-disease-classification/train_images'
images = os.listdir(train_img_path)

print("Training images:", len(images))

In [ ]:
import json
file = open('../input/cassava-leaf-disease-classification/label_num_to_disease_map.json')

class_map = json.load(file)
print(json.dumps(class_map,indent=1))
lis = json.dumps(class_map)

In [ ]:
import pandas as pd

train_file = '../input/cassava-leaf-disease-classification/train.csv'

train = pd.read_csv(train_file)
train.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

ax = plt.figure(figsize=(14,6))

ax = sns.countplot(x = 'label',data=train, palette="pastel")

# Samples of "0": "Cassava Bacterial Blight (CBB)"

In [ ]:
samples = train[train['label']==0].sample(3).reset_index() #Reset index isn't necessary
samples

In [ ]:
plt.figure(figsize=(14,6))

for i, (image_id, label) in enumerate(zip(samples.image_id, samples.label)):
    plt.subplot(1,3,i+1)
    path = os.path.join('../input/cassava-leaf-disease-classification/train_images',image_id)
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
plt.show()

# Samples of "1": "Cassava Brown Streak Disease (CBSD)"

In [ ]:
samples = train[train['label']==1].sample(3)

plt.figure(figsize=(14,6))

for i, (image_id, label) in enumerate(zip(samples.image_id, samples.label)):
    plt.subplot(1,3,i+1)
    path = os.path.join('../input/cassava-leaf-disease-classification/train_images',image_id)
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
plt.show()

# Samples of "2": "Cassava Green Mottle (CGM)"

In [ ]:
samples = train[train['label']==2].sample(3)

plt.figure(figsize=(14,6))

for i, (image_id, label) in enumerate(zip(samples.image_id, samples.label)):
    plt.subplot(1,3,i+1)
    path = os.path.join('../input/cassava-leaf-disease-classification/train_images',image_id)
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
plt.show()

# Samples of "3": "Cassava Mosaic Disease (CMD)"****

In [ ]:
samples = train[train['label']==3].sample(3)

plt.figure(figsize=(14,6))

for i, (image_id, label) in enumerate(zip(samples.image_id, samples.label)):
    plt.subplot(1,3,i+1)
    path = os.path.join('../input/cassava-leaf-disease-classification/train_images',image_id)
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
plt.show()

# Samples of "4": "Healthy"

In [ ]:
samples = train[train['label']==4].sample(3)

plt.figure(figsize=(14,6))

for i, (image_id, label) in enumerate(zip(samples.image_id, samples.label)):
    plt.subplot(1,3,i+1)
    path = os.path.join('../input/cassava-leaf-disease-classification/train_images',image_id)
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    plt.imshow(img)
plt.show()

In [ ]:
train.info()

In [ ]:
# We need to convert labels to string ,label     21397 non-null  int64 

train['label'] = train['label'].astype('str')
train.info()

***Image Data Generator***

In [ ]:
train.head(2)

In [ ]:
img = plt.imread('../input/cassava-leaf-disease-classification/train_images/1000015157.jpg')
img.shape

In [ ]:
classes = 5
img_rows, img_cols = 512, 512
batch_size = 32

STEPS_PER_EPOCH = len(train)*0.7 / batch_size
VALIDATION_STEPS = len(train)*0.3 / batch_size

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

TRAIN_DIR = '../input/cassava-leaf-disease-classification/train_images'

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   rotation_range = 45,
                                   zoom_range = 0.2,
                                   width_shift_range = 0.3,
                                   height_shift_range = 0.3,
                                   shear_range = 0.2,
                                   horizontal_flip = True,
                                   vertical_flip = True,
                                   fill_mode = 'nearest',
                                   validation_split = 0.3)

train_generator = train_datagen.flow_from_dataframe(train,
                                                    TRAIN_DIR,
                                                    subset='training',
                                                    x_col = 'image_id',
                                                    y_col = 'label',
                                                    target_size=(img_rows, img_cols),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

validation_datagen = ImageDataGenerator(validation_split = 0.3)

validation_generator = validation_datagen.flow_from_dataframe(train,
                                                    TRAIN_DIR,
                                                    subset='validation',
                                                    x_col = 'image_id',
                                                    y_col = 'label',
                                                    target_size=(img_rows, img_cols),
                                                    batch_size=batch_size,
                                                    class_mode='categorical')

In [ ]:
epochs = 1

In [ ]:
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, GlobalAveragePooling2D
from tensorflow.keras.layers import Conv2D, MaxPooling2D, ZeroPadding2D
from keras.layers.normalization import BatchNormalization
from tensorflow.keras.models import Model


# Re-loads the MobileNet model without the top or FC layers
vgg16 = VGG16(weights = 'imagenet',include_top = False,input_shape = (img_rows, img_cols, 3))

for layer in vgg16.layers:
    layer.trainable = False
    

def addTopModelvgg16(bottom_model, num_classes):
    """creates the top or head of the model that will be 
    placed ontop of the bottom layers"""

    top_model = bottom_model.output
    top_model = GlobalAveragePooling2D()(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(1024,activation='relu')(top_model)
    top_model = Dense(512,activation='relu')(top_model)
    top_model = Dense(num_classes,activation='softmax')(top_model)
    return top_model


num_classes = classes

FC_Head = addTopModelvgg16(vgg16, num_classes)

model = Model(inputs = vgg16.input, outputs = FC_Head)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
                   
checkpoint = ModelCheckpoint("./vgg19.h5",
                             monitor = "val_loss",
                             mode = "min",
                             save_best_only = True,
                             verbose = 1)

earlystop = EarlyStopping(monitor = 'val_loss', 
                          min_delta = 0.001, 
                          patience = 15,
                          verbose = 1,
                          restore_best_weights = True)

reduce_lr = ReduceLROnPlateau(monitor = 'val_loss',
                              factor = 0.2,
                              patience = 2,
                              verbose = 1,
                              min_delta = 0.0001,
                              min_lr = 0.00000001)


callbacks = [checkpoint,earlystop,reduce_lr]



model.compile(loss = 'categorical_crossentropy',
              optimizer = Adam(),
              metrics = ['accuracy'])


history = model.fit(
    train_generator,
    steps_per_epoch = STEPS_PER_EPOCH,
    epochs = epochs,
    validation_data = validation_generator,
    validation_steps = VALIDATION_STEPS,
    callbacks = callbacks
)

In [ ]:
result = pd.DataFrame(history.history)
result.plot(figsize=(10,8))

In [ ]:
sample = pd.read_csv('../input/cassava-leaf-disease-classification/sample_submission.csv')
sample

In [ ]:
from PIL import Image

test_image = '../input/cassava-leaf-disease-classification/test_images'
preds = []

for image_id in sample['image_id']:
    image = Image.open(os.path.join(test_image, image_id))
    image = image.resize((img_rows, img_cols))
    image = np.expand_dims(image, axis = 0)
    preds.append(np.argmax(model.predict(image)))
    
sample['label'] = preds
sample